# Fast $\log_2$ approximation for trilinear mipmapping

## Some definitions:

- $(u, v)\in [0, 1]^2$: the coordinate of the sample to be taken
- $n$: the size of the full-resolution image.
- $N = \lfloor \log_2 n \rfloor + 1 $: the number of mip-map levels.
- $f_k(u, v)$: the texture function for each mip-map level, with bilinear interpolation of the pixels.
    - $f_0(u, v)$ is the original, full-resolution $n \times n$ pixel image.
    - $f_{N-1}(u, v)$ is the $1 \times 1$ pixel image.
- $w > 0$: the requested texel size, normalized on the texture size. I.e. this is the diameter of the sample to be taken from the texture.
    - $w = 1$ is the full image size.
    - $w = \frac{1}{n}$ is the size of one pixel of the full resolution image.

## Mip-map levels

The number of mip-map levels is:

$$N = \lfloor \log_2 n \rfloor + 1$$

The resolution of each level is, with $k = 0$ the full resolution one:

$$n_k = \lfloor \frac{n} {2^k} \rfloor$$

If $n$ is a power of 2, then this is obviously correct, for example with $n = 16$:

$$\begin{align*}
N &= 4 + 1 = 5\\
n_k &= 16, 8, 4, 2, 1
\end{align*}$$

If $n$ is not a power of 2, then this still works, for example with $n = 15$ [Guthe2005]:

$$\begin{align*}
N &= 3 + 1 = 4\\
n_k &= 15, \lfloor\frac{15}{2}\rfloor = 7, \lfloor\frac{15}{4}\rfloor = 3, \lfloor\frac{15}{8}\rfloor = 1
\end{align*}$$

[Guthe2005] Guthe, S., and P. Heckbert 2005. Non-power-of-two Mipmap creation. NVIDIA Technical Report. https://download.nvidia.com/developer/Papers/2005/NP2_Mipmapping/NP2_Mipmap_Creation.pdf

In [ ]:
import math

for n in range(1, 17):
    N = int(math.floor(math.log2(n))) + 1
    n_k = [int(math.floor(n / (2**k))) for k in range(N)]
    print(f"n={n}: N={N}, n_k={n_k}")


## Trilinear mip-mapping

To select the right mip-map level, we take $\log_2 w$ and we add it to the $1 \times 1$ mip-map level $(N - 1)$.

$$k = (N - 1) + \log_2 w$$

- If $w = 1$, we're basically want to sample the whole image and we need to most coarse mip-map level. Indeed, $\log_2 w = 0$ and we get $k = N - 1$.
- If $w = \frac{1}{n}$, then the sample size is the size of one pixel of the full-resolution image,
  and we want the top-level image. Indeed, $\log_2 w = -(N - 1)$ and we get $k = 0$.

To evaluate $f_k(u, v)$, we blend between two mip-map levels:

$$f_k(u, v) = \begin{cases}
    (1 - \delta k) f_{k_0}(u, v) + \delta k f_{k_1}(u, v) & \text{if } k \in [0, N - 1)\\
    f_0(u, v) & \text{if } k < 0 \\
    f_{N-1}(u, v) & \text{if } k \geq N - 1 \\
\end{cases}$$

with:

$$\begin{align*}k_0 &= \lfloor k \rfloor \\
    k_1 &= \lceil k \rceil \\
    \delta k &= k - k_0
\end{align*}$$

## Fast $\log_2$ approximation

Assume $w > 0$ is a normalized single-precision floating point value. It's binary representation is:

$$w = {-1}^s \cdot 2^e \cdot m$$

with:

- $s \in \{0, 1\}$: the sign bit: 0 if $w > 0$, 1 if $w < 0$
- $e$: integer exponent
- $m \in \left[ 1, 2 \right)$: is the significand.

Because $w > 0$, we can simplify this to:

$$w = 2^e \cdot m$$

Taking $\log_2$ becomes:

$$\begin{align*}\log_2(w) 
    &= \log_2 \left( 2^e \cdot m \right)\\
    &= \log_2 \left( 2^e \right) + \log_2 m\\
    &= e + \log_2 m
\end{align*}$$

We know that $e$ is integer, and:

$$m \in \left[ 1, 2 \right) \\
\Downarrow \\
\log_2 m \in \left[0, 1\right)$$

We finally get that:

$$\begin{align*}k_0 &= e \\
    k_1 &= e + 1 \\
    \delta k &= \log_2 m
\end{align*}$$

Then, we finally make a very crude approximation:

$$\delta k = \log_2 m \approx m - 1$$

It's a very crude approximation as you can see in the plot below.
The maximimum absolute error is 0.086 and the maximum relative error is 31%.
But for blending between to mip-map levels this is perfectly adequate.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

m = np.arange(1, 2, 0.001)
dk = np.log2(m)
dk_fast = m - 1
abs_err = np.abs(dk - dk_fast)

print(f"max absolute error: {np.max(abs_err):.2f}")

plt.plot(m, dk, label="$log_2 m$")
plt.plot(m, dk_fast, label="$m-1$")
plt.plot(m, abs_err, label="abs err")
plt.xlabel("$m$")
plt.legend()
plt.show()

## Implementation

To implement the fast $\log_2$ approximation, we first need to reinterpret the
single precision $w$ as a 32-bit integer `i` [wiki]:

```
b = float2bits(w)
```

- The most significant bit is the sign bit, which is 0 ($w > 0$)
- Then we have 8 exponent bits, which stores the exponent with a bias of 127, thus $e + 127$
- Then the 23 least significant bits store the fraction of the significand $m$, the 1 before
  the decimal point is not stored.

To extract $e$, we can simply bit-shift out the significand and subtract the bias. 
We don't need to make the sign bit, as it's always 0 ($w > 0$):

```
e = (b >> 23) - 127
```

To extract $m$, we set the exponent to zero, and reinterpret the result again as a floating point value.
We first need to mask out the original exponent, and then replace it by the bias 127,
shifted 23 bits to the left:

```
m = bits2float((127 << 23) | (b & 0x7fffff))
```

[wiki] Wikipedia contributors. (2025, January 11). Single-precision floating-point format. In Wikipedia, The Free Encyclopedia. Retrieved 17:57, February 28, 2025, from https://en.wikipedia.org/w/index.php?title=Single-precision_floating-point_format&oldid=1268685998



In [ ]:
import struct


def float2bits(w: float):
    return struct.unpack("I", struct.pack("f", w))[0]


def bits2float(w: float):
    return struct.unpack("f", struct.pack("I", w))[0]


def fast_log2(w: float):
    assert w > 0
    b = float2bits(w)
    e = (b >> 23) - 127
    m = bits2float((127 << 23) | (b & 0x7FFFFF))
    return e + (m - 1)

In [ ]:
w = np.arange(0.1, 1, 0.001)

k = np.log2(w)
k_fast = [fast_log2(x) for x in w]

plt.plot(w, k, label="log2")
plt.plot(w, k_fast, label="fastlog2")
plt.xlabel("$w$")
plt.legend()
plt.show()

plt.plot(w, np.abs(k - k_fast), label="abs err")
plt.legend()
plt.xlabel("$w$")
plt.show()